In [18]:
pip install windows-tools

In [1]:
import requests
from windows_tools.installed_software import get_installed_software  # Función para obtener software instalado en Windows






# Definir el programa que deseas buscar
programa = "Riot games"

# URL de la API de Vulners para buscar información sobre el programa
url = "https://vulners.com/api/v3/search/lucene/"

# Parámetros de la solicitud
params = {
    "query": programa,
    "size": 10,  # Número máximo de resultados que quieres obtener
    "sort": "cvss.score"  # Ordenar por puntuación CVSS
}

# Realizar la solicitud GET a la API
response = requests.get(url, params=params)

# Comprobar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Convertir la respuesta JSON en un diccionario de Python
    data = response.json()
    
    # Iterar sobre los resultados y extraer los campos deseados
    for resultado in data["data"]["search"]:
        # Obtener el ID del resultado
        id_resultado = resultado["_id"]
        
        # Extraer los campos deseados, manejando los casos donde las claves no están presentes
        titulo = resultado["_source"].get("title")
        cve_list = resultado["_source"].get("cvelist")
        cvss_score = resultado["_source"]["cvss"].get("score")
        cvss2_access_complexity = resultado["_source"]["cvss2"]["cvssV2"].get("accessComplexity") if resultado["_source"].get("cvss2") else None
        cvss2_base_score = resultado["_source"]["cvss2"]["cvssV2"].get("baseScore") if resultado["_source"].get("cvss2") else None
        cvss3_attack_complexity = resultado["_source"]["cvss3"]["cvssV3"].get("attackComplexity") if resultado["_source"].get("cvss3") else None
        cvss3_base_score = resultado["_source"]["cvss3"]["cvssV3"].get("baseScore") if resultado["_source"].get("cvss3") else None
        fecha_publicacion_str = resultado["_source"].get("published")

        # Imprimir los campos extraídos junto con el ID del resultado
        print("ID:", id_resultado)
        print("Titulo:", titulo)
        print("CVE List:", cve_list)
        print("CVSS Score:", cvss_score)
        ### Imprimir mejor la fecha de publicación
        if fecha_publicacion_str:
            fecha_publicacion = datetime.fromisoformat(fecha_publicacion_str)
            # Formatear la fecha en un formato más legible
            fecha_publicacion_formatted = fecha_publicacion.strftime("%Y-%m-%d %H:%M:%S")
            print("Fecha de publicación:", fecha_publicacion_formatted)
        else:
            print("No se encontró fecha de publicación para esta entrada.")



        
        print()
        
        # Pequeño bloque para poder investigar CVE
        # Pequeño bloque para poder investigar CVE
        for cve_id in cve_list:
            # Asigno CVE
            url_mitre = f'https://cveawg.mitre.org/api/cve/{cve_id}'
            response2 = requests.get(url_mitre)
            params = {}
            
            if response2.status_code == 200:
                print("Validando CVE en Mitre")
                # Convertir la respuesta JSON en un diccionario de Python
                cve_data = response2.json()  # Cambio de nombre de la variable
                containers = cve_data.get('containers', {})
                cna = containers.get('cna', {})
                affected = cna.get('affected', [])
            
                if affected:
                    # Verificar si hay información de impacto
                    metrics_list = cve_data.get('containers', {}).get('cna', {}).get('metrics', [])
                    if metrics_list:
                        for item in affected:
                            product = item.get('product', '')
                            vendor = item.get('vendor', '')
                            versions = [v.get('version', '') for v in item.get('versions', [])]
                            state = cve_data.get('cveMetadata', {}).get('state', '')
                            for metric in metrics_list:
                                other_content = metric.get('other', {})
                                if other_content:
                                    impacto = other_content.get('content', {}).get('other')
                                    if impacto:
                                        print("Impacto:", impacto)
                            print('Product:', product)
                            print('Vendor:', vendor)
                            print('Versions:', versions)
                            print('state:', state)
                            print()
                    else:
                        print(f"No se encontró información de impacto para la CVE {cve_id}.")
                else:
                    print('No se encontraron datos de afectación para la CVE especificada.')
            else:
                # Imprimir un mensaje de error si la solicitud no fue exitosa
                print(f"Error al obtener los datos de la CVE {cve_id}. Código de estado:", response2.status_code)


#####Fin
else:
    # Imprimir un mensaje de error si la solicitud no fue exitosa
    print("Error al obtener los datos. Código de estado:", response.status_code)


ID: THREATPOST:065F7608AC06475E765018E97F14998D
Titulo: 'Appalling' Riot Games Job Fraud Takes Aim at Wallets
CVE List: ['CVE-2021-44228']
CVSS Score: 9.3
Fecha de publicación: 2021-12-10 19:00:36

Validando CVE en Mitre
Impacto: critical
Product: Apache Log4j2
Vendor: Apache Software Foundation
Versions: ['2.0-beta9']
state: PUBLISHED

ID: THREATPOST:4DF584EB3FA47CA6245D964EA2A1A2FB
Titulo: Mojang Resets Users' Passwords, Microsoft Insists Not a Hack
CVE List: ['CVE-2017-11882']
CVSS Score: 9.3
Fecha de publicación: 2015-01-22 14:35:45

Validando CVE en Mitre
No se encontró información de impacto para la CVE CVE-2017-11882.
ID: SECURELIST:9881BE178E7DC46C87F3498AE0FCB86C
Titulo: Gaming-related cyberthreats in 2023: Minecrafters targeted the most
CVE List: []
CVSS Score: 0.0
Fecha de publicación: 2023-11-06 10:00:58

ID: TALOSBLOG:A33C23F55B9DD5987EBF9E5FC5555BFB
Titulo: Threat Source newsletter (Feb. 16, 2023) — Recapping what we may have missed so far this year
CVE List: ['CVE-2023-2